In [13]:
!touch .gitignore
!echo "savant_pitch_level.csv" >> .gitignore
!cat .gitignore

savant_pitch_level.csv
savant_pitch_level.csv
savant_pitch_level.csv
savant_pitch_level.csv


In [15]:
!git add .
!git commit -m "added ease of life git features"
!git push

[main 13b7eb4] added ease of life git features
 3 files changed, 1082 insertions(+), 145 deletions(-)
 create mode 100644 .gitignore
 create mode 100644 .ipynb_checkpoints/EDA-checkpoint.ipynb
Enumerating objects: 8, done.
Counting objects: 100% (8/8), done.
Delta compression using up to 8 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (6/6), 6.10 KiB | 6.10 MiB/s, done.
Total 6 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/RudimentaryChef/RedsHackathon.git
   65aebdd..13b7eb4  main -> main


In [1]:
# Data Manipulation and Analysis
import pandas as pd
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Statistical Modeling and Machine Learning
from sklearn import model_selection, metrics, linear_model, ensemble
import statsmodels.api as sm
from scipy import stats
#excel files
import openpyxl




In [2]:
fangraphs_data = pd.read_csv("fangraphs_season_level.csv")
savant_data = pd.read_csv("savant_pitch_level.csv")

In [3]:
fangraphs_data.head()

,PlayerId,MLBAMID,Name,NameASCII,Throws,Season,Age,Team,Role,G,...,Pit_plus_FC,Stf_plus_FS,Loc_plus_FS,Pit_plus_FS,Stf_plus_FO,Loc_plus_FO,Pit_plus_FO,Stuff_plus,Location_plus,Pitching_plus
0,18,491703,NeftalÃ­ Feliz,Neftali Feliz,R,2021,33,- - -,RP,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.316418,95.123881,95.873134
1,1157,488984,Tommy Hunter,Tommy Hunter,R,2021,34,NYM,RP,3,...,103.100000,NaN,NaN,NaN,NaN,NaN,NaN,86.285556,94.884444,95.112222
2,1159,461865,Andrew Romine,Andrew Romine,R,2021,35,CHC,RP,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.925000,108.425000,99.200000
3,1246,456713,Matt Bush,Matt Bush,R,2021,35,TEX,RP,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,124.852174,98.340580,102.611594
4,1247,493603,Adam Ottavino,Adam Ottavino,R,2021,35,BOS,RP,69,...,66.583333,NaN,NaN,NaN,NaN,NaN,NaN,116.269527,94.625394,102.238617


In [4]:
print(fangraphs_data.columns)


Index(['PlayerId', 'MLBAMID', 'Name', 'NameASCII', 'Throws', 'Season', 'Age',
       'Team', 'Role', 'G',
       ...
       'Pit_plus_FC', 'Stf_plus_FS', 'Loc_plus_FS', 'Pit_plus_FS',
       'Stf_plus_FO', 'Loc_plus_FO', 'Pit_plus_FO', 'Stuff_plus',
       'Location_plus', 'Pitching_plus'],
      dtype='object', length=308)


In [5]:
fangraphs_columns = ['Name','PlayerId', 'Throws', 'Age', 'Team', 'Role', 'ERA', 'G', 'GS', 
                   'IP', 'SO', 'BB', 'SV', 'HLD', 'FIP', 'xFIP', 'SIERA', 'WHIP']

fangraphs_data = fangraphs_data[fangraphs_columns]

In [6]:
savant_data.columns
savant_columns = ['player_name', 'pitch_type', 'release_speed', 'release_spin_rate', 
                  'events', 'pfx_x', 'pfx_z', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 
                  'spin_axis', 'game_year']

savant_data = savant_data[savant_columns]

In [7]:
savant_data['player_name'] = savant_data['player_name'].str.split(', ').str[::-1].str.join(' ')
savant_data['player_name'] = savant_data['player_name'].astype(str)
fangraphs_data['Name'] = fangraphs_data['Name'].astype(str)


merged_data = pd.merge(savant_data, fangraphs_data, left_on='player_name', right_on='Name')


In [8]:
merged_data.head()

,player_name,pitch_type,release_speed,release_spin_rate,events,pfx_x,pfx_z,vx0,vy0,vz0,...,GS,IP,SO,BB,SV,HLD,FIP,xFIP,SIERA,WHIP
0,Danny Duffy,FF,93.2,2351.0,strikeout,0.43,1.57,-3.263176,-135.520600,-8.932598,...,0,1.0,1,0,0,0,1.170030,1.170030,-0.479878,0.000000
1,Danny Duffy,FF,93.2,2351.0,strikeout,0.43,1.57,-3.263176,-135.520600,-8.932598,...,12,60.0,64,22,0,0,3.436697,4.283335,4.179673,1.233333
2,Danny Duffy,SL,82.9,2408.0,NaN,-0.42,-0.34,-1.343797,-120.739091,-0.762147,...,0,1.0,1,0,0,0,1.170030,1.170030,-0.479878,0.000000
3,Danny Duffy,SL,82.9,2408.0,NaN,-0.42,-0.34,-1.343797,-120.739091,-0.762147,...,12,60.0,64,22,0,0,3.436697,4.283335,4.179673,1.233333
4,Danny Duffy,FF,93.6,2233.0,NaN,0.37,1.53,-6.325447,-136.141440,-4.315313,...,0,1.0,1,0,0,0,1.170030,1.170030,-0.479878,0.000000


In [9]:
merged_data

,player_name,pitch_type,release_speed,release_spin_rate,events,pfx_x,pfx_z,vx0,vy0,vz0,...,GS,IP,SO,BB,SV,HLD,FIP,xFIP,SIERA,WHIP
0,Danny Duffy,FF,93.2,2351.0,strikeout,0.43,1.57,-3.263176,-135.520600,-8.932598,...,0,1.0,1,0,0,0,1.170030,1.170030,-0.479878,0.000000
1,Danny Duffy,FF,93.2,2351.0,strikeout,0.43,1.57,-3.263176,-135.520600,-8.932598,...,12,60.0,64,22,0,0,3.436697,4.283335,4.179673,1.233333
2,Danny Duffy,SL,82.9,2408.0,NaN,-0.42,-0.34,-1.343797,-120.739091,-0.762147,...,0,1.0,1,0,0,0,1.170030,1.170030,-0.479878,0.000000
3,Danny Duffy,SL,82.9,2408.0,NaN,-0.42,-0.34,-1.343797,-120.739091,-0.762147,...,12,60.0,64,22,0,0,3.436697,4.283335,4.179673,1.233333
4,Danny Duffy,FF,93.6,2233.0,NaN,0.37,1.53,-6.325447,-136.141440,-4.315313,...,0,1.0,1,0,0,0,1.170030,1.170030,-0.479878,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6643763,Wes Parsons,CU,83.2,2560.0,NaN,0.69,-1.13,1.331086,-121.142400,-1.867732,...,1,4.0,3,3,0,0,11.255040,7.648407,6.291218,3.250000
6643764,Wes Parsons,FF,95.4,2241.0,NaN,-0.61,1.43,8.088776,-138.489000,-8.332467,...,1,4.0,3,3,0,0,11.255040,7.648407,6.291218,3.250000
6643765,Wes Parsons,FF,93.8,2211.0,NaN,-0.75,1.29,8.375425,-136.238300,-6.811934,...,1,4.0,3,3,0,0,11.255040,7.648407,6.291218,3.250000
6643766,Wes Parsons,FF,93.1,2239.0,NaN,-0.51,1.36,8.523849,-135.237200,-7.403496,...,1,4.0,3,3,0,0,11.255040,7.648407,6.291218,3.250000
